# Детекция дыма и огня

In [ ]:
# импорты
# системные библиотеки
import os
import json

# работа с данными
import pandas as pd
import numpy as np

# работа с изображениями
import matplotlib.pyplot as plt
import cv2

# машинное обучение
from ultralytics import YOLO

# метрики
from sklearn.metrics import precision_score, recall_score

In [3]:
# загрузка данных
with open('data/train/annotations/instances_default.json', 'r') as file:
    json_train = json.load(file)

with open('data/val/annotations/instances_default.json', 'r') as file:
    json_val = json.load(file)

In [4]:
# загрузка данных
paths_train = os.listdir('data/train/images/train')
paths_val = os.listdir('data/val/images/val')

In [5]:
# преобразование в таблицы
images_train_pd = pd.DataFrame(json_train['images'])[['file_name', 'id']]
annotations_train_pd = pd.DataFrame(json_train['annotations'])[['image_id', 'category_id']]
annotations_train_pd.columns = ['id', 'category_id']
train_paths = pd.merge(images_train_pd, annotations_train_pd, on='id').drop_duplicates(subset='id')

images_val_pd = pd.DataFrame(json_val['images'])[['file_name', 'id']]
annotations_val_pd = pd.DataFrame(json_val['annotations'])[['image_id', 'category_id']]
annotations_val_pd.columns = ['id', 'category_id']
val_paths = pd.merge(images_val_pd, annotations_val_pd, on='id').drop_duplicates(subset='id')

train_paths.shape, val_paths.shape

((1142, 3), (308, 3))

In [6]:
# получение изображений и меток
images_train = []
labels_train = []
for i, path in enumerate(train_paths['file_name']):
    try:
        image = cv2.imread('data/train/images/' + path)
        image = cv2.resize(image, (128,128))
        images_train.append(image)

        labels_train.append(train_paths['category_id'].iloc[i])
    except:
        pass
    
images_train = np.array(images_train, dtype='float32')/255.

images_val = []
labels_val = []
for i, path in enumerate(val_paths['file_name']):
    try:
        image = cv2.imread('data/val/images/' + path)
        image = cv2.resize(image, (128,128))
        images_val.append(image)

        labels_val.append(val_paths['category_id'].iloc[i])
    except:
        pass
    
images_val = np.array(images_val, dtype='float32')/255.

images_train.shape, len(labels_train), images_val.shape, len(labels_val)

((1142, 128, 128, 3), 1142, (308, 128, 128, 3), 308)

In [7]:
# преобразование меток для обучения
labels_train_fire = []
for i in labels_train:
    if i == 5:
        labels_train_fire.append([1, 0])
    else:
        labels_train_fire.append([0, 1])
labels_train_fire = np.array(labels_train_fire, dtype='float32')

labels_val_fire = []
for i in labels_val:
    if i == 5:
        labels_val_fire.append([1, 0])
    else:
        labels_val_fire.append([0, 1])
labels_val_fire = np.array(labels_val_fire, dtype='float32')

labels_train_smoke = []
for i in labels_train:
    if i == 4:
        labels_train_smoke.append([1, 0])
    else:
        labels_train_smoke.append([0, 1])
labels_train_smoke = np.array(labels_train_smoke, dtype='float32')

labels_val_smoke = []
for i in labels_val:
    if i == 4:
        labels_val_smoke.append([1, 0])
    else:
        labels_val_smoke.append([0, 1])
labels_val_smoke = np.array(labels_val_smoke, dtype='float32')

In [19]:
# распределение изображений по папкам для огня
for i, image in enumerate(images_train):
    if labels_train_fire[i][0] == 1:
        cv2.imwrite(f"fire/train/fire/{i}.jpeg", image)
    else:
        cv2.imwrite(f"fire/train/no_fire/{i}.jpeg", image)

for i, image in enumerate(images_val):
    if labels_val_fire[i][0] == 1:
        cv2.imwrite(f"fire/val/fire/{i}.jpeg", image)
    else:
        cv2.imwrite(f"fire/val/no_fire/{i}.jpeg", image)

In [20]:
# распределение изображений по папкам для дыма
for i, image in enumerate(images_train):
    if labels_train_smoke[i][0] == 1:
        cv2.imwrite(f"smoke/train/smoke/{i}.jpeg", image)
    else:
        cv2.imwrite(f"smoke/train/no_smoke/{i}.jpeg", image)

for i, image in enumerate(images_val):
    if labels_val_smoke[i][0] == 1:
        cv2.imwrite(f"smoke/val/smoke/{i}.jpeg", image)
    else:
        cv2.imwrite(f"smoke/val/no_smoke/{i}.jpeg", image)

In [23]:
# создание модели
model = YOLO('yolov8n-cls.pt')

# обучение
model.train(data='fire', epochs=50, batch=32)

New https://pypi.org/project/ultralytics/8.3.119 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.118  Python-3.12.4 torch-2.5.1+cu118 CUDA:0 (GeForce GTX 1050, 2048MiB)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=fire, epochs=50, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning C:\repo\cv_fire_detection\fire\train... 1142 images, 0 corrupt: 100%|██████████| 1142/1142 [00:02<00:00, 522.90it/s]


train: New cache created: C:\repo\cv_fire_detection\fire\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 0.10.0 MB/s, size: 1.1 KB)


val: Scanning C:\repo\cv_fire_detection\fire\val... 308 images, 0 corrupt: 100%|██████████| 308/308 [00:00<00:00, 327.39it/s]


val: New cache created: C:\repo\cv_fire_detection\fire\val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train2
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all      0.477          1



50 epochs completed in 0.095 hours.
Optimizer stripped from runs\classify\train2\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train2\weights\best.pt, 3.0MB

Validating runs\classify\train2\weights\best.pt...
Ultralytics 8.3.118  Python-3.12.4 torch-2.5.1+cu118 CUDA:0 (GeForce GTX 1050, 2048MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: C:\repo\cv_fire_detection\fire\train... found 1142 images in 2 classes  
val: C:\repo\cv_fire_detection\fire\val... found 308 images in 2 classes  
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00,  8.15it/s]


                   all      0.562          1
Speed: 0.7ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E201ED84A0>
curves: []
curves_results: []
fitness: 0.7808441519737244
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5616883039474487, 'metrics/accuracy_top5': 1.0, 'fitness': 0.7808441519737244}
save_dir: WindowsPath('runs/classify/train2')
speed: {'preprocess': 0.7030350651649467, 'inference': 0.984713636443031, 'loss': 0.00732207811413357, 'postprocess': 0.006355194721897071}
task: 'classify'
top1: 0.5616883039474487
top5: 1.0

In [24]:
# валидация
model.val()

Ultralytics 8.3.118  Python-3.12.4 torch-2.5.1+cu118 CUDA:0 (GeForce GTX 1050, 2048MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: C:\repo\cv_fire_detection\fire\train... found 1142 images in 2 classes  
val: C:\repo\cv_fire_detection\fire\val... found 308 images in 2 classes  
test: None...
val: Fast image access  (ping: 0.00.0 ms, read: 10.62.3 MB/s, size: 1.1 KB)


val: Scanning C:\repo\cv_fire_detection\fire\val... 308 images, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


                   all      0.565          1
Speed: 0.6ms preprocess, 1.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train22


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E2041C5D60>
curves: []
curves_results: []
fitness: 0.7824675440788269
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5649350881576538, 'metrics/accuracy_top5': 1.0, 'fitness': 0.7824675440788269}
save_dir: WindowsPath('runs/classify/train22')
speed: {'preprocess': 0.5579025975228953, 'inference': 1.1584012986118482, 'loss': 0.015788311713266296, 'postprocess': 0.02221818176149572}
task: 'classify'
top1: 0.5649350881576538
top5: 1.0